In [9]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 3

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Saving vectors of label - 'cat':   0%|          | 1/1733 [00:00<05:18,  5.44it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving vectors of label - 'happy': 100%|██████████| 1742/1742 [00:07<00:00, 245.57it/s]


In [11]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [12]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/50
3112/3112 [==============================] - 3s 973us/step - loss: 1.5756 - acc: 0.4126 - val_loss: 0.8376 - val_acc: 0.6137
Epoch 2/50
3112/3112 [==============================] - 2s 748us/step - loss: 0.7628 - acc: 0.6681 - val_loss: 0.5120 - val_acc: 0.8078
Epoch 3/50
3112/3112 [==============================] - 2s 712us/step - loss: 0.5426 - acc: 0.7857 - val_loss: 0.3703 - val_acc: 0.8646
Epoch 4/50
3112/3112 [==============================] - 2s 771us/step - loss: 0.4191 - acc: 0.8393 - val_loss: 0.3240 - val_acc: 0.8781
Epoch 5/50
3112/3112 [==============================] - 2s 776us/step - loss: 0.3748 - acc: 0.8618 - val_loss: 0.2747 - val_acc: 0.9046
Epoch 6/50
3112/3112 [==============================] - 2s 760us/step - loss: 0.3017 - acc: 0.8891 - val_loss: 0.5409 - val_acc: 0.8030
Epoch 7/50
3112/3112 [==============================] - 2s 793us/step - loss: 0.2586 - acc: 0.9129 - val_loss: 0.2365 - val_acc: 0.9152


## Prediction

In [18]:
print(predict('./data/happy/0ac15fe9_nohash_0.wav', model=model))

happy


In [19]:
model.get_session().graph

AttributeError: 'Sequential' object has no attribute 'get_session'

In [21]:

keras.get_session()

AttributeError: module 'keras' has no attribute 'get_session'

In [25]:
from keras import backend as K
K.get_session().graph